In [3]:
# cluster_seat_allocation.py

# Step 1: Import Libraries and Set Up
import pandas as pd
import numpy as np
import os
from collections import deque

# Ensure output directory exists (one level up in the root directory)
os.makedirs("../data/output", exist_ok=True)

# Debug: Print the current working directory
print("Current working directory:", os.getcwd())

# Step 2: Define Seat Allocation Function with BFS
def allocate_seats_clustered(chairs_df, num_students, image_name, adjacency_threshold=100):
    """
    Allocate seats to students in a clustered format using BFS.
    
    Parameters:
    - chairs_df: DataFrame containing chair information (Seat ID, Status, center_x, center_y, etc.)
    - num_students: Number of students to allocate seats to
    - image_name: Name of the image (for saving the Excel file)
    - adjacency_threshold: Maximum distance (in pixels) between chair centers to consider them adjacent
    
    Returns:
    - assignments: List of seat assignment messages
    - updated_chairs_df: Updated DataFrame with new seat statuses
    """
    chairs_df = chairs_df.copy()
    assignments = []

    # Step 2.1: Build an adjacency graph of chairs
    num_chairs = len(chairs_df)
    adj_list = [[] for _ in range(num_chairs)]
    
    # Compute pairwise distances between chairs and build the graph
    for i in range(num_chairs):
        for j in range(i + 1, num_chairs):
            chair_i = chairs_df.iloc[i]
            chair_j = chairs_df.iloc[j]
            distance = np.sqrt((chair_i["center_x"] - chair_j["center_x"])**2 + (chair_i["center_y"] - chair_j["center_y"])**2)
            if distance < adjacency_threshold:
                adj_list[i].append(j)
                adj_list[j].append(i)

    # Step 2.2: Use BFS to find the nearest empty seats to occupied seats
    occupied_seats = chairs_df[chairs_df["Status"] == "Occupied"].index.tolist()
    if not occupied_seats:
        # If no seats are occupied, start with the first empty seat
        empty_seats = chairs_df[chairs_df["Status"] == "Empty"].index.tolist()
        if not empty_seats:
            return "No empty seats available.", chairs_df
        first_seat = empty_seats[0]
        chairs_df.loc[first_seat, "Status"] = "Occupied"
        assignments.append(f"Student 1 → {chairs_df.loc[first_seat, 'Seat ID']}")
        occupied_seats = [first_seat]
        num_students -= 1

    # Step 2.3: BFS to allocate remaining students
    for student in range(1, num_students + 1):
        # Initialize BFS queue with currently occupied seats
        queue = deque(occupied_seats)
        visited = set(occupied_seats)
        found_seat = None

        # BFS to find the nearest empty seat
        while queue and found_seat is None:
            current = queue.popleft()
            # Explore neighbors of the current chair
            for neighbor in adj_list[current]:
                if neighbor in visited:
                    continue
                visited.add(neighbor)
                if chairs_df.loc[neighbor, "Status"] == "Empty":
                    found_seat = neighbor
                    break
                queue.append(neighbor)

        # If no empty seat is found in the cluster, look for any remaining empty seat
        if found_seat is None:
            empty_seats = chairs_df[chairs_df["Status"] == "Empty"].index.tolist()
            if not empty_seats:
                assignments.append("No more empty seats available.")
                break
            found_seat = empty_seats[0]

        # Allocate the seat
        chairs_df.loc[found_seat, "Status"] = "Occupied"
        assignments.append(f"Student {student + 1 if occupied_seats else student} → {chairs_df.loc[found_seat, 'Seat ID']}")
        occupied_seats.append(found_seat)

    # Save the updated seat status to Excel
    chairs_df.to_excel(f"../data/output/seat_status_clustered_{image_name.split('.')[0]}.xlsx", index=False)
    return assignments, chairs_df

# Step 3: Main Function to Process Images
def main(num_students=3, adjacency_threshold=100):
    # Define the input and output directories (relative path from src/ to root)
    input_dir = "../data/output"
    images_dir = "../data/images"

    # Get all image files in the images directory
    image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpeg', '.jpg', '.png'))]
    if not image_files:
        print("No images found in ../data/images/ directory.")
        return

    for image_name in image_files:
        print(f"\nProcessing {image_name}...\n" + "="*50)

        # Construct the path to the initial seat status Excel file
        initial_status_file = f"seat_status_{image_name.split('.')[0]}.xlsx"
        initial_status_path = os.path.join(input_dir, initial_status_file)

        # Check if the initial seat status file exists
        if not os.path.exists(initial_status_path):
            print(f"Initial seat status file not found for {image_name}: {initial_status_path}")
            continue

        # Read the initial seat status
        chairs_df = pd.read_excel(initial_status_path)

        # Allocate seats using BFS clustering
        assignments, updated_chairs_df = allocate_seats_clustered(chairs_df, num_students, image_name, adjacency_threshold)

        # Print the results
        print(f"Initial Seat Status for {image_name}:\n", chairs_df[["Seat ID", "Status"]])
        print(f"Seat Assignments for {image_name}:\n", "\n".join(assignments))
        print(f"Updated Seat Status for {image_name}:\n", updated_chairs_df[["Seat ID", "Status"]])

# Run the main function
if __name__ == "__main__":
    num_students = 3  # Number of students to allocate seats to
    adjacency_threshold = 100  # Distance threshold (pixels) for considering chairs adjacent
    main(num_students=num_students, adjacency_threshold=adjacency_threshold)

Current working directory: /home/shreyx/Desktop/Projects/NNDL Mini Project/Automated-Seat-allocation/src

Processing classroom_image7.jpeg...
Initial Seat Status for classroom_image7.jpeg:
     Seat ID Status
0    Seat 1  Empty
1    Seat 2  Empty
2    Seat 3  Empty
3    Seat 4  Empty
4    Seat 5  Empty
5    Seat 6  Empty
6    Seat 7  Empty
7    Seat 8  Empty
8    Seat 9  Empty
9   Seat 10  Empty
10  Seat 11  Empty
11  Seat 12  Empty
12  Seat 13  Empty
13  Seat 14  Empty
14  Seat 15  Empty
15  Seat 16  Empty
16  Seat 17  Empty
17  Seat 18  Empty
18  Seat 19  Empty
19  Seat 20  Empty
20  Seat 21  Empty
21  Seat 22  Empty
22  Seat 23  Empty
23  Seat 24  Empty
24  Seat 25  Empty
Seat Assignments for classroom_image7.jpeg:
 Student 1 → Seat 1
Student 2 → Seat 5
Student 3 → Seat 11
Updated Seat Status for classroom_image7.jpeg:
     Seat ID    Status
0    Seat 1  Occupied
1    Seat 2     Empty
2    Seat 3     Empty
3    Seat 4     Empty
4    Seat 5  Occupied
5    Seat 6     Empty
6    Seat 7

In [4]:
# Step 1: Import Libraries and Set Up
import pandas as pd
import numpy as np
import os
from collections import deque
import cv2
import matplotlib.pyplot as plt

# Ensure output directory exists (one level up in the root directory)
os.makedirs("../data/output", exist_ok=True)

# Debug: Print the current working directory
print("Current working directory:", os.getcwd())

Current working directory: /home/shreyx/Desktop/Projects/NNDL Mini Project/Automated-Seat-allocation/src


In [5]:
# Step 2: Define Seat Allocation Function with BFS
def allocate_seats_clustered(chairs_df, num_students, image_name, adjacency_threshold=100):
    """
    Allocate seats to students in a clustered format using BFS.
    
    Parameters:
    - chairs_df: DataFrame containing chair information (Seat ID, Status, center_x, center_y, etc.)
    - num_students: Number of students to allocate seats to
    - image_name: Name of the image (for saving the Excel file)
    - adjacency_threshold: Maximum distance (in pixels) between chair centers to consider them adjacent
    
    Returns:
    - assignments: List of seat assignment messages
    - updated_chairs_df: Updated DataFrame with new seat statuses
    - newly_allocated_seats: List of indices of newly allocated seats
    """
    chairs_df = chairs_df.copy()
    assignments = []
    newly_allocated_seats = []

    # Step 2.1: Build an adjacency graph of chairs
    num_chairs = len(chairs_df)
    adj_list = [[] for _ in range(num_chairs)]
    
    # Compute pairwise distances between chairs and build the graph
    for i in range(num_chairs):
        for j in range(i + 1, num_chairs):
            chair_i = chairs_df.iloc[i]
            chair_j = chairs_df.iloc[j]
            distance = np.sqrt((chair_i["center_x"] - chair_j["center_x"])**2 + (chair_i["center_y"] - chair_j["center_y"])**2)
            if distance < adjacency_threshold:
                adj_list[i].append(j)
                adj_list[j].append(i)

    # Step 2.2: Use BFS to find the nearest empty seats to occupied seats
    occupied_seats = chairs_df[chairs_df["Status"] == "Occupied"].index.tolist()
    if not occupied_seats:
        # If no seats are occupied, start with the first empty seat
        empty_seats = chairs_df[chairs_df["Status"] == "Empty"].index.tolist()
        if not empty_seats:
            return "No empty seats available.", chairs_df, newly_allocated_seats
        first_seat = empty_seats[0]
        chairs_df.loc[first_seat, "Status"] = "Occupied"
        assignments.append(f"Student 1 → {chairs_df.loc[first_seat, 'Seat ID']}")
        newly_allocated_seats.append(first_seat)
        occupied_seats = [first_seat]
        num_students -= 1

    # Step 2.3: BFS to allocate remaining students
    for student in range(1, num_students + 1):
        # Initialize BFS queue with currently occupied seats
        queue = deque(occupied_seats)
        visited = set(occupied_seats)
        found_seat = None

        # BFS to find the nearest empty seat
        while queue and found_seat is None:
            current = queue.popleft()
            # Explore neighbors of the current chair
            for neighbor in adj_list[current]:
                if neighbor in visited:
                    continue
                visited.add(neighbor)
                if chairs_df.loc[neighbor, "Status"] == "Empty":
                    found_seat = neighbor
                    break
                queue.append(neighbor)

        # If no empty seat is found in the cluster, look for any remaining empty seat
        if found_seat is None:
            empty_seats = chairs_df[chairs_df["Status"] == "Empty"].index.tolist()
            if not empty_seats:
                assignments.append("No more empty seats available.")
                break
            found_seat = empty_seats[0]

        # Allocate the seat
        chairs_df.loc[found_seat, "Status"] = "Occupied"
        assignments.append(f"Student {student + 1 if occupied_seats else student} → {chairs_df.loc[found_seat, 'Seat ID']}")
        newly_allocated_seats.append(found_seat)
        occupied_seats.append(found_seat)

    # Save the updated seat status to Excel
    chairs_df.to_excel(f"../data/output/seat_status_clustered_{image_name.split('.')[0]}.xlsx", index=False)
    return assignments, chairs_df, newly_allocated_seats

In [6]:
# Step 3: Define Visualization Function
def visualize_seats(image_path, chairs_df, newly_allocated_seats, image_name):
    """
    Visualize the seats on the image, marking initial statuses and newly allocated seats.
    
    Parameters:
    - image_path: Path to the original image
    - chairs_df: DataFrame containing chair information
    - newly_allocated_seats: List of indices of newly allocated seats
    - image_name: Name of the image (for saving the visualization)
    """
    # Load the image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Could not load image at {image_path}. Skipping visualization.")
        return

    # Create a copy of the image for marking
    marked_img = img.copy()

    # Mark all chairs based on their initial status and whether they were newly allocated
    for idx, chair in chairs_df.iterrows():
        x1, y1, x2, y2 = int(chair["x1"]), int(chair["y1"]), int(chair["x2"]), int(chair["y2"])
        center_x, center_y = int(chair["center_x"]), int(chair["center_y"])
        radius = int((x2 - x1) / 2)  # Approximate radius for the circle

        if idx in newly_allocated_seats:
            # Blue circle for newly allocated seats
            cv2.circle(marked_img, (center_x, center_y), radius, (255, 0, 0), 2)  # Blue in BGR
            cv2.putText(marked_img, f"{chair['Seat ID']} (New)", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        elif chair["Status"] == "Occupied":
            # Red circle for initially occupied seats
            cv2.circle(marked_img, (center_x, center_y), radius, (0, 0, 255), 2)  # Red in BGR
            cv2.putText(marked_img, f"{chair['Seat ID']} (Occupied)", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        else:
            # Green circle for empty seats
            cv2.circle(marked_img, (center_x, center_y), radius, (0, 255, 0), 2)  # Green in BGR
            cv2.putText(marked_img, f"{chair['Seat ID']} (Empty)", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Convert BGR (OpenCV) to RGB (Matplotlib)
    original_img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    marked_img_rgb = cv2.cvtColor(marked_img, cv2.COLOR_BGR2RGB)

    # Create a figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    ax1.imshow(original_img_rgb)
    ax1.set_title(f"Original Image: {image_name}")
    ax1.axis("off")
    ax2.imshow(marked_img_rgb)
    ax2.set_title(f"Clustered Allocation: {image_name}\n(Empty: Green, Occupied: Red, New: Blue)")
    ax2.axis("off")

    # Save the visualization
    plt.savefig(f"../data/output/clustered_visualization_{image_name.split('.')[0]}.png", bbox_inches="tight")
    plt.close()

    # Optional: Display the image if running in an environment with a GUI
    # cv2.imshow("Clustered Seat Allocation", marked_img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

In [7]:
# Step 4: Main Function to Process Images
def main(num_students=3, adjacency_threshold=100):
    # Define the input and output directories (relative path from src/ to root)
    input_dir = "../data/output"
    images_dir = "../data/images"

    # Get all image files in the images directory
    image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpeg', '.jpg', '.png'))]
    if not image_files:
        print("No images found in ../data/images/ directory.")
        return

    for image_name in image_files:
        print(f"\nProcessing {image_name}...\n" + "="*50)

        # Construct the path to the initial seat status Excel file
        initial_status_file = f"seat_status_{image_name.split('.')[0]}.xlsx"
        initial_status_path = os.path.join(input_dir, initial_status_file)

        # Check if the initial seat status file exists
        if not os.path.exists(initial_status_path):
            print(f"Initial seat status file not found for {image_name}: {initial_status_path}")
            continue

        # Construct the path to the original image
        image_path = os.path.join(images_dir, image_name)

        # Read the initial seat status
        chairs_df = pd.read_excel(initial_status_path)

        # Allocate seats using BFS clustering
        assignments, updated_chairs_df, newly_allocated_seats = allocate_seats_clustered(chairs_df, num_students, image_name, adjacency_threshold)

        # Print the results
        print(f"Initial Seat Status for {image_name}:\n", chairs_df[["Seat ID", "Status"]])
        print(f"Seat Assignments for {image_name}:\n", "\n".join(assignments))
        print(f"Updated Seat Status for {image_name}:\n", updated_chairs_df[["Seat ID", "Status"]])

        # Visualize the seats
        visualize_seats(image_path, updated_chairs_df, newly_allocated_seats, image_name)

# Run the main function
if __name__ == "__main__":
    num_students = 3  # Number of students to allocate seats to
    adjacency_threshold = 100  # Distance threshold (pixels) for considering chairs adjacent
    main(num_students=num_students, adjacency_threshold=adjacency_threshold)


Processing classroom_image7.jpeg...
Initial Seat Status for classroom_image7.jpeg:
     Seat ID Status
0    Seat 1  Empty
1    Seat 2  Empty
2    Seat 3  Empty
3    Seat 4  Empty
4    Seat 5  Empty
5    Seat 6  Empty
6    Seat 7  Empty
7    Seat 8  Empty
8    Seat 9  Empty
9   Seat 10  Empty
10  Seat 11  Empty
11  Seat 12  Empty
12  Seat 13  Empty
13  Seat 14  Empty
14  Seat 15  Empty
15  Seat 16  Empty
16  Seat 17  Empty
17  Seat 18  Empty
18  Seat 19  Empty
19  Seat 20  Empty
20  Seat 21  Empty
21  Seat 22  Empty
22  Seat 23  Empty
23  Seat 24  Empty
24  Seat 25  Empty
Seat Assignments for classroom_image7.jpeg:
 Student 1 → Seat 1
Student 2 → Seat 5
Student 3 → Seat 11
Updated Seat Status for classroom_image7.jpeg:
     Seat ID    Status
0    Seat 1  Occupied
1    Seat 2     Empty
2    Seat 3     Empty
3    Seat 4     Empty
4    Seat 5  Occupied
5    Seat 6     Empty
6    Seat 7     Empty
7    Seat 8     Empty
8    Seat 9     Empty
9   Seat 10     Empty
10  Seat 11  Occupied
11  Se

In [8]:
# cluster_seat_allocation.py

# Step 1: Import Libraries and Set Up
import pandas as pd
import numpy as np
import os
from collections import deque
import cv2
import matplotlib.pyplot as plt

# Ensure output directory exists (one level up in the root directory)
os.makedirs("../data/output", exist_ok=True)

# Debug: Print the current working directory
print("Current working directory:", os.getcwd())

# Step 2: Define Seat Allocation Function with BFS
def allocate_seats_clustered(chairs_df, num_students, image_name, adjacency_threshold=100):
    """
    Allocate seats to students in a clustered format using BFS.
    
    Parameters:
    - chairs_df: DataFrame containing chair information (Seat ID, Status, center_x, center_y, etc.)
    - num_students: Number of students to allocate seats to
    - image_name: Name of the image (for saving the Excel file)
    - adjacency_threshold: Maximum distance (in pixels) between chair centers to consider them adjacent
    
    Returns:
    - assignments: List of seat assignment messages
    - updated_chairs_df: Updated DataFrame with new seat statuses
    - newly_allocated_seats: List of indices of newly allocated seats
    """
    chairs_df = chairs_df.copy()
    assignments = []
    newly_allocated_seats = []

    # Step 2.1: Build an adjacency graph of chairs
    num_chairs = len(chairs_df)
    adj_list = [[] for _ in range(num_chairs)]
    
    # Compute pairwise distances between chairs and build the graph
    for i in range(num_chairs):
        for j in range(i + 1, num_chairs):
            chair_i = chairs_df.iloc[i]
            chair_j = chairs_df.iloc[j]
            distance = np.sqrt((chair_i["center_x"] - chair_j["center_x"])**2 + (chair_i["center_y"] - chair_j["center_y"])**2)
            if distance < adjacency_threshold:
                adj_list[i].append(j)
                adj_list[j].append(i)

    # Step 2.2: Use BFS to find the nearest empty seats to occupied seats
    occupied_seats = chairs_df[chairs_df["Status"] == "Occupied"].index.tolist()
    if not occupied_seats:
        # If no seats are occupied, start with the first empty seat
        empty_seats = chairs_df[chairs_df["Status"] == "Empty"].index.tolist()
        if not empty_seats:
            return "No empty seats available.", chairs_df, newly_allocated_seats
        first_seat = empty_seats[0]
        chairs_df.loc[first_seat, "Status"] = "Occupied"
        assignments.append(f"Student 1 → {chairs_df.loc[first_seat, 'Seat ID']}")
        newly_allocated_seats.append(first_seat)
        occupied_seats = [first_seat]
        num_students -= 1

    # Step 2.3: BFS to allocate remaining students
    for student in range(1, num_students + 1):
        # Initialize BFS queue with currently occupied seats
        queue = deque(occupied_seats)
        visited = set(occupied_seats)
        found_seat = None

        # BFS to find the nearest empty seat
        while queue and found_seat is None:
            current = queue.popleft()
            # Explore neighbors of the current chair
            for neighbor in adj_list[current]:
                if neighbor in visited:
                    continue
                visited.add(neighbor)
                if chairs_df.loc[neighbor, "Status"] == "Empty":
                    found_seat = neighbor
                    break
                queue.append(neighbor)

        # If no empty seat is found in the cluster, look for any remaining empty seat
        if found_seat is None:
            empty_seats = chairs_df[chairs_df["Status"] == "Empty"].index.tolist()
            if not empty_seats:
                assignments.append("No more empty seats available.")
                break
            found_seat = empty_seats[0]

        # Allocate the seat
        chairs_df.loc[found_seat, "Status"] = "Occupied"
        assignments.append(f"Student {student + 1 if occupied_seats else student} → {chairs_df.loc[found_seat, 'Seat ID']}")
        newly_allocated_seats.append(found_seat)
        occupied_seats.append(found_seat)

    # Save the updated seat status to Excel
    output_excel_path = f"../data/output/seat_status_clustered_{image_name.split('.')[0]}.xlsx"
    print(f"Saving updated seat status to: {output_excel_path}")
    chairs_df.to_excel(output_excel_path, index=False)
    return assignments, chairs_df, newly_allocated_seats

# Step 3: Define Visualization Function
def visualize_seats(image_path, chairs_df, newly_allocated_seats, image_name):
    """
    Visualize the seats on the image, marking initial statuses and newly allocated seats.
    
    Parameters:
    - image_path: Path to the original image
    - chairs_df: DataFrame containing chair information
    - newly_allocated_seats: List of indices of newly allocated seats
    - image_name: Name of the image (for saving the visualization)
    """
    print(f"Attempting to load image from: {image_path}")
    # Load the image
    img = cv2.imread(image_path)
    if img is None:
        print(f"Failed to load image at {image_path}. Skipping visualization.")
        return

    print("Image loaded successfully. Creating visualization...")
    # Create a copy of the image for marking
    marked_img = img.copy()

    # Mark all chairs based on their initial status and whether they were newly allocated
    for idx, chair in chairs_df.iterrows():
        x1, y1, x2, y2 = int(chair["x1"]), int(chair["y1"]), int(chair["x2"]), int(chair["y2"])
        center_x, center_y = int(chair["center_x"]), int(chair["center_y"])
        radius = int((x2 - x1) / 2)  # Approximate radius for the circle

        if idx in newly_allocated_seats:
            # Blue circle for newly allocated seats
            cv2.circle(marked_img, (center_x, center_y), radius, (255, 0, 0), 2)  # Blue in BGR
            cv2.putText(marked_img, f"{chair['Seat ID']} (New)", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)
        elif chair["Status"] == "Occupied":
            # Red circle for initially occupied seats
            cv2.circle(marked_img, (center_x, center_y), radius, (0, 0, 255), 2)  # Red in BGR
            cv2.putText(marked_img, f"{chair['Seat ID']} (Occupied)", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 2)
        else:
            # Green circle for empty seats
            cv2.circle(marked_img, (center_x, center_y), radius, (0, 255, 0), 2)  # Green in BGR
            cv2.putText(marked_img, f"{chair['Seat ID']} (Empty)", (x1, y1-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

    # Convert BGR (OpenCV) to RGB (Matplotlib)
    original_img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    marked_img_rgb = cv2.cvtColor(marked_img, cv2.COLOR_BGR2RGB)

    # Create a figure with two subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 6))
    ax1.imshow(original_img_rgb)
    ax1.set_title(f"Original Image: {image_name}")
    ax1.axis("off")
    ax2.imshow(marked_img_rgb)
    ax2.set_title(f"Clustered Allocation: {image_name}\n(Empty: Green, Occupied: Red, New: Blue)")
    ax2.axis("off")

    # Save the visualization
    output_image_path = f"../data/output/clustered_visualization_{image_name.split('.')[0]}.png"
    print(f"Saving visualization to: {output_image_path}")
    try:
        plt.savefig(output_image_path, bbox_inches="tight")
        print(f"Visualization saved successfully to {output_image_path}")
    except Exception as e:
        print(f"Failed to save visualization: {e}")
    finally:
        plt.close()

    # Optional: Display the image if running in an environment with a GUI
    # print("Attempting to display the image interactively...")
    # cv2.imshow("Clustered Seat Allocation", marked_img)
    # cv2.waitKey(0)
    # cv2.destroyAllWindows()

# Step 4: Main Function to Process Images
def main(num_students=3, adjacency_threshold=100):
    # Define the input and output directories (relative path from src/ to root)
    input_dir = "../data/output"
    images_dir = "../data/images"

    # Debug: Check if the directories exist
    print(f"Input directory (for Excel files): {input_dir}")
    print(f"Images directory: {images_dir}")
    if not os.path.exists(input_dir):
        print(f"Input directory {input_dir} does not exist. Exiting.")
        return
    if not os.path.exists(images_dir):
        print(f"Images directory {images_dir} does not exist. Exiting.")
        return

    # Get all image files in the images directory
    image_files = [f for f in os.listdir(images_dir) if f.endswith(('.jpeg', '.jpg', '.png'))]
    if not image_files:
        print("No images found in ../data/images/ directory.")
        return

    print(f"Found {len(image_files)} images: {image_files}")

    for image_name in image_files:
        print(f"\nProcessing {image_name}...\n" + "="*50)

        # Construct the path to the initial seat status Excel file
        initial_status_file = f"seat_status_{image_name.split('.')[0]}.xlsx"
        initial_status_path = os.path.join(input_dir, initial_status_file)

        # Check if the initial seat status file exists
        if not os.path.exists(initial_status_path):
            print(f"Initial seat status file not found for {image_name}: {initial_status_path}")
            continue

        # Construct the path to the original image
        image_path = os.path.join(images_dir, image_name)
        if not os.path.exists(image_path):
            print(f"Image file not found: {image_path}")
            continue

        # Read the initial seat status
        print(f"Reading initial seat status from: {initial_status_path}")
        chairs_df = pd.read_excel(initial_status_path)

        # Allocate seats using BFS clustering
        print("Allocating seats...")
        assignments, updated_chairs_df, newly_allocated_seats = allocate_seats_clustered(chairs_df, num_students, image_name, adjacency_threshold)

        # Print the results
        print(f"Initial Seat Status for {image_name}:\n", chairs_df[["Seat ID", "Status"]])
        print(f"Seat Assignments for {image_name}:\n", "\n".join(assignments))
        print(f"Updated Seat Status for {image_name}:\n", updated_chairs_df[["Seat ID", "Status"]])

        # Visualize the seats
        print("Generating visualization...")
        visualize_seats(image_path, updated_chairs_df, newly_allocated_seats, image_name)

# Run the main function
if __name__ == "__main__":
    num_students = 3  # Number of students to allocate seats to
    adjacency_threshold = 100  # Distance threshold (pixels) for considering chairs adjacent
    main(num_students=num_students, adjacency_threshold=adjacency_threshold)

Current working directory: /home/shreyx/Desktop/Projects/NNDL Mini Project/Automated-Seat-allocation/src
Input directory (for Excel files): ../data/output
Images directory: ../data/images
Found 14 images: ['classroom_image7.jpeg', 'classroom_image11.jpeg', 'classroom_image14.jpeg', 'classroom_image9.jpeg', 'classroom_image2.jpeg', 'classroom_image10.jpeg', 'classroom_image13.jpeg', 'classroom_image6.jpeg', 'classroom_image5.jpeg', 'classroom_image1.jpeg', 'classroom_image8.jpeg', 'classroom_image3.jpeg', 'classroom_image4.jpeg', 'classroom_image12.jpeg']

Processing classroom_image7.jpeg...
Reading initial seat status from: ../data/output/seat_status_classroom_image7.xlsx
Allocating seats...
Saving updated seat status to: ../data/output/seat_status_clustered_classroom_image7.xlsx
Initial Seat Status for classroom_image7.jpeg:
     Seat ID Status
0    Seat 1  Empty
1    Seat 2  Empty
2    Seat 3  Empty
3    Seat 4  Empty
4    Seat 5  Empty
5    Seat 6  Empty
6    Seat 7  Empty
7    Seat